# A Dual Approach to Holistic Regression

## 2021.01.25 — Numerical Approach

In [1]:
using Random, Distributions
using LinearAlgebra
using Gurobi, JuMP
using FiniteDifferences

In [2]:
# Create a gurobi model without the annoying academic license message
gurobi_env = Gurobi.Env()
function create_gurobi_model(; TimeLimit=-1, LogFile="logs.txt")
    model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env)));
    if TimeLimit >= 0
        println("Set Gurobi TimeLimit.")
        set_optimizer_attribute(model, "TimeLimit", TimeLimit)
    end
    set_optimizer_attribute(model, "LogFile", LogFile)
    set_optimizer_attribute(model, "OutputFlag", 0)
    return model
end;

Academic license - for non-commercial use only


### 1. Data

In [26]:
Random.seed!(2021)

# Params
ϵ = 10^(-15)
α = 0.05
γ = 1
n, p = 100, 5
t_α = quantile(TDist(n-p), 1 - α/2) # Beware: n-p-1 if we add intercept
#t_α = 1
# Data
X = rand(n, p)
y = rand(n)

# Variance
M = X'X
M_inv = M^-1
σ_tilde =  sqrt((y'*(I - X*M_inv*X')*y)/(n-p))
σ_X = σ_tilde * sqrt.(diag(M_inv))

;

In [27]:
function get_max_inner(s_plus, s_minus)
    # Get s
    s = s_plus + s_minus
    
    # Compute matrices
    Z = Diagonal(s)
    Z_plus = Diagonal(s_plus)
    Z_minus = Diagonal(s_minus)
    
    # Compute D
    D = (I/γ + Z*M)^-1

    # Compute u_Z
    function compute_u_Z(λ_plus, λ_minus, μ)
        Z_minus*(λ_minus + μ) - Z_plus*(λ_plus + μ)
    end

    # Compute norm
    function compute_DZ_square_norm(in_norm)
        return in_norm' * D*Z * in_norm
    end
    
    # Compute max
    model = create_gurobi_model()

    @variable(model, λ_plus[1:p] >= 0)
    @variable(model, λ_minus[1:p] >= 0)
    @variable(model, μ[1:p] >= 0)

    obj_1 = ϵ*sum(s_plus.*λ_plus + s_minus.*λ_minus)
    obj_2 = t_α*μ'*(s.*σ_X)
    obj_3 = - 0.5 * compute_DZ_square_norm(X'y - compute_u_Z(λ_plus, λ_minus, μ))

    @objective(model, Max, obj_1 + obj_2 + obj_3)

    optimize!(model)
    
    return objective_value(model)
    
end

get_max_inner (generic function with 1 method)

In [29]:
#(e -> get_max_inner(e[1:p], e[p+1:end]))([rand() for e in 1:2p])
for i in 1:1
    s = [rand() for e in 1:p]
    s_plus = [rand([0,1])*e for e in s]
    s_minus = s - s_plus
    println(s)
    println(s_plus)
    println(s_minus)
    println(get_max_inner(s_plus, s_minus))
end

[0.596923, 0.0408826, 0.632922, 0.348465, 0.395823]
[0.0, 0.0408826, 0.632922, 0.0, 0.395823]
[0.596923, 0.0, 0.0, 0.348465, 0.0]
-6.97916120715078


In [30]:
grad(central_fdm(5, 1), e -> get_max_inner(e[1:p], e[p+1:end]), convert(Array{Float64, 1},[s_plus; s_minus]))

UndefVarError: UndefVarError: s_plus not defined